In [1]:
using PauliPropagation
using Plots
using ReverseDiff
using ReverseDiff: GradientTape, gradient!, compile, gradient
using Bits
using Random
using Distributions: Uniform
using NLopt

### GS of TFIM using Pauli Propagation
- idea: initialise using DQA
- then: do a few loops of adaptVQE for the final steps
- problem: if we prepropagate, adapt-VQEs position sandwiches around U^\dagger H U, so its like adaptVQE acts on plus first, then on the U from the interpolating Hamiltonian 

### AdaptVQE (tiles approach)

In [2]:
function tfim_1d_obc(nq::Int;topology = nothing, gamma = 1.0)

    if isnothing(topology)
        topology = bricklayertopology(nq;periodic=false) #chain with obc
    end

    H = PauliSum(nq)
    for qind in 1:nq
        add!(H, :X, qind, gamma)
    end
    for pair in topology
        add!(H, [:Z, :Z], collect(pair), 1.0)
    end
    return H
end

tfim_1d_obc (generic function with 1 method)

In [3]:
function neel_bits(nq::Int; up_on_odd::Bool=true)
    if up_on_odd
        # |0101...> → "1" on even sites
        return collect(2:2:nq)
    else
        # |1010...> → "1" on odd sites
        return collect(1:2:nq)
    end
end

neel_bits (generic function with 1 method)

In [4]:
function overlapwithneel(operator, nq::Int; 
                           up_on_odd::Bool=true, 
                           params=nothing)
                           
    # Create Néel state bit representation: indices of "1" bits
    nb = neel_bits(nq; up_on_odd=up_on_odd)
    
    # Compute overlap with computational basis state
    return overlapwithcomputational(operator, nb)
end

overlapwithneel (generic function with 1 method)

In [5]:
function generate_full_bit_pool(nq::Int)
    # Choose appropriate UInt type based on number of qubits (same as PP does)
    # Work with bit representations directly (easier to manipulate)
    UIntType = if nq <= 4
        UInt8
    elseif nq <= 8
        UInt16
    elseif nq <= 16
        UInt32
    else
        UInt64
    end
    
    # Here, we generate all non-identity bit patterns (therefore -1), which gives the full operator pool.
    pool = UIntType[]
    for i in 1:(4^nq - 1)
        push!(pool, UIntType(i))
    end
    
    return pool
end

generate_full_bit_pool (generic function with 1 method)

In [6]:
# Function to convert bit representation to PauliString when needed
function bit_to_paulistring(bit_repr, nq)
    paulis = Symbol[]
    sites = Int[]
    
    for qubit in 1:nq #going from right to left in bitstring
        pauli_val = getpauli(bit_repr, qubit) # get pauli of qubit as 0,1,2,3
        #println("Qubit $qubit: Pauli value = $pauli_val")
        if pauli_val != 0  # Skip identity (0) since Paulis are initialised as identity by default
            pauli_symbol = [:I, :X, :Y, :Z][pauli_val + 1] # julia indexing starts at 1!
            push!(paulis, pauli_symbol)
            push!(sites, qubit)
        end
    end
    
    return PauliString(nq, paulis, sites, 1.0), paulis, sites
end

bit_to_paulistring (generic function with 1 method)

In [7]:
"""
    calc_gradients(bit_pool, H, nq;
                   circuit=nothing, params=nothing,
                   tol=1e-12, verbose=false, up_on_odd=true)

ADAPT-VQE gradients with in-place propagation:
g_P = ⟨φ₀ | [U† H U, U† P U] i | φ₀⟩, where |φ₀⟩ is the Néel state
selected by `up_on_odd`.
"""
function calc_gradients(bit_pool, H, nq;
                        circuit::Union{Nothing,Any}=nothing,
                        params::Union{Nothing,AbstractVector}=nothing,
                        tol::Float64=1e-12,
                        verbose::Bool=false,
                        up_on_odd::Bool=true)

    grads = Float64[]

    # Pre-propagate H once: H_prop = U† H U (only if a circuit is provided)
    H_prop = H
    if circuit !== nothing
        H_prop = deepcopy(H)                 # avoid mutating H
        propagate!(circuit, H_prop, params)  # in-place propagation
    end

    # For each pool element, pre-propagate P (to P_prop) and form [H_prop, P_prop]
    for (k, bit_repr) in enumerate(bit_pool)
        P= bit_to_paulistring(bit_repr, nq)[1]
        psum = PauliSum(nq)
        P = add!(psum, P) # PP works with PauliSums (see datatypes example notebook)

        P_prop = P
        if circuit !== nothing
            P_prop = deepcopy(P)                 # avoid mutating the pool op
            propagate!(circuit, P_prop, params)  # in-place propagation
        end
        
        C = commutator(H_prop, P_prop) 
        if !(iterate(C) !== nothing)           # if the commutator is empty 
            verbose && println("op[$k]: ", P, "  commutator=0  → grad=0.0")
            push!(grads, 0.0)
            continue
        end
        g = overlapwithneel(im * C, nq; up_on_odd=up_on_odd)
        if abs(imag(g)) > tol
            @warn "Gradient has non-negligible imaginary part" imag=imag(g) op=P
        end

        push!(grads, real(g))
        verbose && println("op[$k]: ", P, "  grad=", real(g))
    end

    return grads
end

calc_gradients

In [8]:
function pick_top_operator(gradients::AbstractVector, operators::AbstractVector; rng=Random.GLOBAL_RNG)
    length(gradients) == length(operators) || throw(ArgumentError("gradients and operators must have same length"))
    isempty(gradients) && throw(ArgumentError("gradients must not be empty"))
    
    mags = abs.(gradients)
    order = sortperm(mags, rev=true)  # indices sorted by |grad| descending (following the nature paper implementation)
    gradients_sorted = gradients[order]
    operators_sorted = operators[order]

    max_mag = mags[order[1]]
    tied_top = filter(i -> mags[i] == max_mag, order)

    chosen_idx = rand(rng, tied_top)

    return operators[chosen_idx], gradients[chosen_idx], gradients_sorted, operators_sorted
end

pick_top_operator (generic function with 1 method)

In [9]:
function check_convergence(gradients; tol=1e-4)
    max_grad = maximum(abs.(gradients))
    return max_grad < tol, max_grad
end

check_convergence (generic function with 1 method)

In [10]:
function pauli_rotation_from_bits(bit_repr, nq)
    _, paulis, sites = bit_to_paulistring(bit_repr, nq) 
    return PauliRotation(paulis, sites)
end

pauli_rotation_from_bits (generic function with 1 method)

In [11]:
function append_from_bits!(circuit, thetas, chose_op, nq; theta_init=rand())
    gate = pauli_rotation_from_bits(chose_op, nq)
    pushfirst!(circuit, gate)
    pushfirst!(thetas, theta_init)
    return circuit, thetas    # index of the new parameter
end

append_from_bits! (generic function with 1 method)

In [13]:
function fulllossfunction(thetas, circuit, nq;gamma=1.0,topology = nothing, max_freq = Inf, max_weight = Inf)

    if isnothing(topology)
        topology = bricklayertopology(nq;periodic=false) #chain with obc
    end

    # differentiation libraries use custom types to trace through the computation
    # we need to make all of our objects typed like that so that nothing breaks
    CoeffType = eltype(thetas)

    H = PauliSum(CoeffType, nq) #define Hamiltonian again (otherwise Reverse Diff breaks)
    
    for qind in 1:nq
        add!(H, :X, qind, CoeffType(gamma))
    end
    for pair in topology
        add!(H, [:Z, :Z], collect(pair), CoeffType(1.0))
    end

    # wrap the coefficients into PauliFreqTracker so that we can use `max_freq` truncation.
    # usually this happens automatically but the in-place propagate!() function does not allow that.
    wrapped_H = wrapcoefficients(H, PauliFreqTracker)
    
    # we also need to run the in-place version with `!`, because by default we copy the Pauli sum
    output_H = propagate!(circuit, wrapped_H, thetas; max_freq, max_weight);
    return overlapwithplus(output_H)
end

fulllossfunction (generic function with 1 method)

In [14]:
# Adam optimizer
function adam(thetas_init, closed_lossfunction, nq; eta=0.02, steps=100)
    nparams = length(thetas_init)

 
   # ReverseDiff tape on the tied loss (same as GD system)
    tape = GradientTape(closed_lossfunction, thetas_init)
    compiled_tape = compile(tape)

    # Buffers & Adam state
    grad = similar(thetas_init)
    thetas = copy(thetas_init)
    opt_energy = Vector{Float64}(undef, 0)

    m = zero(thetas); v = zero(thetas)
    β1, β2, ϵ = 0.9, 0.999, 1e-8

    for t in 1:steps
        gradient!(grad, tape, thetas)

        # Adam update
        m .= β1 .* m .+ (1 - β1) .* grad
        v .= β2 .* v .+ (1 - β2) .* (grad .^ 2)
        mhat = m ./ (1 - β1^t)
        vhat = v ./ (1 - β2^t)
        thetas .-= eta .* mhat ./ (sqrt.(vhat) .+ ϵ)

        # Log E/Q using the tied view
        push!(opt_energy, closed_lossfunction(thetas)/nq)
    end
    return thetas, opt_energy
end


adam (generic function with 1 method)

In [ ]:
# L-BFGS optimizer using NLopt package
# https://nlopt.readthedocs.io/en/latest/NLopt_Reference/
function lbfgs_nlopt(thetas_init, closed_lossfunction, nq;
                     max_iters=500, xtol_rel=1e-8, ftol_rel=1e-12,
                     max_time=Inf,
                     lower_bounds=nothing, upper_bounds=nothing,
                     verbose=false)

    n  = length(thetas_init)

    tape     = GradientTape(closed_lossfunction, thetas_init) #assume thetas_init is already Float64
    compiled_tape = compile(tape)
    gbuf     = zeros(n) # gradient buffer (preallocated), we need this for type stability between NLopt (C library) and Julia
    opt_energy = Float64[]
    evals = Ref(0)

    opt = Opt(:LD_LBFGS, n) # NLopt package for L-BFGS algorithm
    NLopt.maxeval!(opt, max_iters) # maximum number of function evaluations
    #NLopt.xtol_rel!(opt, xtol_rel) # relative tolerance on optimization parameters (x axis) # this can cause stall at first step of optim
    #NLopt.ftol_rel!(opt, ftol_rel) # relative tolerance on function value (f(x) axis)
    NLopt.maxtime!(opt, isfinite(max_time) ? max_time : 0.0) # maximum time in seconds (0.0 means no cap)

    # using these causes an error! (we're bounding the thetas during the adaptVQE anyways, so this is not necessarily needed)
    if lower_bounds !== nothing
        NLopt.lower_bounds!(opt, Float64.(lower_bounds))
    end
    if upper_bounds !== nothing
        NLopt.upper_bounds!(opt, Float64.(upper_bounds))
    end

    NLopt.min_objective!(opt, (x, grad) -> begin # grad is write only that belongs to NLopt, sometimes NLopt wont ask for it, then it passes an empty array
        fx = closed_lossfunction(x) :: Float64
        if !isempty(grad)
            gradient!(gbuf, compiled_tape, x) #ReverseDiff writes gradient into gbuf, then we copy into grad
            @inbounds @simd for i in 1:n
                grad[i] = gbuf[i]
            end
        end
        evals[] += 1
        push!(opt_energy, fx)
        return fx
    end)

    minf, thetas, ret = NLopt.optimize(opt, thetas_init) # minf is best function / objective value found, thetas is the best params (that give minf), ret is termination code.
    if verbose
        @info "NLopt LBFGS terminated" status=ret evals=evals[] fmin=minf
    end

    return thetas, opt_energy/nq # LBFGS can do multiple function evaluations per iteration, so len(opt_energy) may be > max_iters.
end


lbfgs_nlopt (generic function with 1 method)

In [16]:
function target_optimization(nq, circuit, thetas; max_freq=Inf, max_weight=Inf, verbose = false)
    # again, automatic diff example notebook for more details
    closed_lossfunction = let const_nq=nq, const_max_freq=max_freq, const_max_weight=max_weight
        # these are now all captured variables with known types and we return a function that only takes thetas s.t. ReverseDiff works
        # See also "closures"
        theta -> fulllossfunction(theta, circuit, const_nq; max_freq=const_max_freq, max_weight=const_max_weight)
    end
    
    opt_thetas, opt_energy_gd = lbfgs_nlopt(thetas,closed_lossfunction,nq)
    if verbose
        println("Optimized thetas: ", opt_thetas)
        println("Optimized energy per qubit: ", opt_energy_gd[end])

        plot(opt_energy_gd)
        display(plot!(title = "Energy optimisation", xlabel = "runs", ylabel = "E/Q"))
    end

    return opt_thetas, opt_energy_gd
end

target_optimization (generic function with 1 method)

In [17]:
function tfim_interpolation_circuit(nq, P; del_t= 0.1, J=1.0, G=1.0, hbar=1.0, topology=nothing, periodic = false) 
    # drives from -X to target + X+ ZZ (TFIM)
    tau = del_t * P # tau is total annealing time, divided into P steps/layers of circuit
    if topology === nothing
        topology = bricklayertopology(nq; periodic=periodic)
    end

    circuit = tfitrottercircuit(nq, P; topology=topology, start_with_ZZ=false) # 1st order trotter
    periodic ? nbonds = nq : nbonds = nq - 1 # if periodic nq bonds, else nq-1 (1d chain)
    total = P * (nq + nbonds)
    params  = Vector{Float64}(undef, total)
    idx = 1
    
    for m in 1:P # layers of trotterized circuit
        s_m = (m - 0.5) / P

        gamma_m = J* s_m * del_t /hbar
        #println(gamma_m)   # paper Eq. (12)
        beta_m = G*del_t*(2*s_m-1)/ hbar #(1-s_m) * del_t / hbar # paper Eq. (13a) - 1st order trotter
        #println(beta_m)
        theta_X = 2*beta_m
        ##println(theta_X)              # angle for e^{-i theta_X X_j/2} - PauliRotation implementation in PP
        theta_ZZ = 2*gamma_m              # angle for e^{-i theta_ZZ Z_j Z_{j+1}/2}, J=1
        ##println(theta_ZZ)
        @inbounds fill!(view(params, idx:idx+nq-1), theta_X);             idx += nq # set X angles,then shift the index
        #println(params)
        @inbounds fill!(view(params, idx:idx+nbonds-1), theta_ZZ);       idx += nbonds
        #println(params)
    end
    # freeze the circuit to the params
    circuit = freeze(circuit, params)
    return circuit
end


tfim_interpolation_circuit (generic function with 1 method)

In [18]:
# check datatypes
circ = tfim_interpolation_circuit(3,500;topology=bricklayertopology(3))

2500-element Vector{Gate}:
 FrozenGate(PauliRotation([:X], [1]), parameter = -0.2)
 FrozenGate(PauliRotation([:X], [2]), parameter = -0.2)
 FrozenGate(PauliRotation([:X], [3]), parameter = -0.2)
 FrozenGate(PauliRotation([:Z, :Z], [1, 2]), parameter = 0.0002)
 FrozenGate(PauliRotation([:Z, :Z], [2, 3]), parameter = 0.0002)
 FrozenGate(PauliRotation([:X], [1]), parameter = -0.199)
 FrozenGate(PauliRotation([:X], [2]), parameter = -0.199)
 FrozenGate(PauliRotation([:X], [3]), parameter = -0.199)
 FrozenGate(PauliRotation([:Z, :Z], [1, 2]), parameter = 0.0006)
 FrozenGate(PauliRotation([:Z, :Z], [2, 3]), parameter = 0.0006)
 ⋮
 FrozenGate(PauliRotation([:X], [2]), parameter = 0.199)
 FrozenGate(PauliRotation([:X], [3]), parameter = 0.199)
 FrozenGate(PauliRotation([:Z, :Z], [1, 2]), parameter = 0.199)
 FrozenGate(PauliRotation([:Z, :Z], [2, 3]), parameter = 0.199)
 FrozenGate(PauliRotation([:X], [1]), parameter = 0.2)
 FrozenGate(PauliRotation([:X], [2]), parameter = 0.2)
 FrozenGate(Paul

In [19]:
function adaptVQE_loop(nq, hamiltonian, bit_pool; theta_init = rand(Uniform(-π, π)), max_iters=10, conv_tol=1e-4, interpol_circ=nothing, verbose=true)
    circuit = Any[]      # initialize the adaptVQE circuit
    thetas  = Float64[]; # initialize the parameter list
    energy_per_loop = Float64[];
    max_grads = Float64[] # collect for analysis
    chosen_ops = UInt[]

    for iter in 1:max_iters

        # 1) Calculate gradients
        grads = calc_gradients(bit_pool, hamiltonian, nq; circuit=circuit, params=thetas, tol=1e-12, verbose=false, up_on_odd=true)

        # 2) Check convergence
        converged, max_grad = check_convergence(grads; tol=conv_tol)
        push!(max_grads, max_grad)
        
        if converged
            println("Convergence achieved with max gradient < $conv_tol")
            break
        end

        # 3) Pick top operator
        chose_op, grad_op, grads_sorted, _ = pick_top_operator(grads, bit_pool)

        push!(chosen_ops, chose_op)

        # 4) Append operator to circuit
        circuit, thetas = append_from_bits!(circuit, thetas, chose_op, nq; theta_init=theta_init) # theta_init is multiplied by 2 in append function

        ### DQA initialisation (append the frozen interpolation circuit, which we the overlap with teh driver gs (here plus))
        if interpol_circ != nothing
            append!(circuit, interpol_circ)
        end 
        # 5) Optimize parameters
        thetas, opt_energy_gd = target_optimization(nq, circuit, thetas; verbose=verbose)
        # wrap the thetas to [-π,π] since they are angles
        thetas = rem2pi.(thetas,RoundNearest) # does not seem to help the optimization
        push!(energy_per_loop, opt_energy_gd[end])
    end

    # 6) Plot the gradient convergence
    if verbose
        plot(max_grads, marker=:o)
        display(plot!(title = "Max Gradient Convergence", xlabel = "Iteration", ylabel = "Max Gradient"))
    end

    return circuit, thetas, chosen_ops, energy_per_loop, max_grads
end

adaptVQE_loop (generic function with 1 method)

In [20]:
function scaled_pool_selection(nq, hamiltonian, full_bit_pool; num_runs=5, run_iters=5, conv_tol=1e-4, verbose=false)
    all_chosen_ops = UInt[]
    for run in 1:num_runs
        #println("=== ADAPT-VQE Run $run ===")
        circuit, thetas, chosen_ops = adaptVQE_loop(nq, hamiltonian, full_bit_pool; max_iters=run_iters, conv_tol=conv_tol, verbose=verbose)
        append!(all_chosen_ops, chosen_ops)
    end
    return unique(all_chosen_ops)  # Return unique operators from all runs
end


function scaled_pool_generation(tile_nq, scaled_nq,hamiltonian, full_bit_pool; num_runs=5, run_iters=5, conv_tol=1e-4, verbose=false)
    #Choose appropriate UInt type based on number of qubits (same as PP does)
        UIntType = if scaled_nq <= 4
            UInt8
        elseif scaled_nq <= 8
            UInt16
        elseif scaled_nq <= 16
            UInt32
        else
            UInt64
        end
        
        chosen_ops = scaled_pool_selection(tile_nq, hamiltonian, full_bit_pool; num_runs=num_runs, run_iters=run_iters, conv_tol=conv_tol, verbose=verbose)
        println("Chosen operators from tile ADAPT-VQE: ", length(chosen_ops))
        scaled_pool = UIntType[]
        for op in chosen_ops
            ps, paulis, sites = bit_to_paulistring(op, tile_nq) 
            #println("Scaling operator: ", ps)
            #println("Paulis: ", paulis, " Sites: ", sites)
            weight = length(paulis)
            #println("Weight: ", weight)
            # Preserve the operator's internal geometry: compute zero-based relative offsets
            min_site = minimum(sites)
            #println("Min site: ", min_site)
            rel_sites = sites .- min_site
            #println("Relative sites: ", rel_sites)
            block_span = maximum(rel_sites) + 1 # equivalent to block_span = maximum(sites) - minimum(sites) + 1
            #println("Block span: ", block_span)
        # Slide the block from left to right without wrapping (open boundaries)
        for start in 1:(scaled_nq - block_span + 1)
            #println("Start position: ", start)
            new_sites = start .+ rel_sites  # keep internal offsets
            #println("New sites: ", new_sites)
            new_op = PauliString(scaled_nq, paulis, collect(new_sites), 1.0)
            push!(scaled_pool, (new_op.term))
        end
    end
    return unique(scaled_pool)
end

scaled_pool_generation (generic function with 1 method)

### Full-pool test

In [21]:
nq = 4
P = 100
interpol_circ = tfim_interpolation_circuit(nq, P)
hamiltonian = tfim_1d_obc(nq, topology=bricklayertopology(nq;periodic=false))
full_bit_pool = generate_full_bit_pool(nq)
adapt_circuit, adapt_thetas, chosen_ops, energy_per_loop, max_grads = adaptVQE_loop(nq, hamiltonian, full_bit_pool;interpol_circ=interpol_circ, theta_init = rand(Uniform(-pi,pi)), max_iters=25, conv_tol=1e-4, verbose=false);

In [22]:
println(energy_per_loop[end]) #final energy per qubit
println(max_grads[end]) #final max gradient

-0.837614609325684
3.5203036614530503


### Scaled pool test

In [23]:
# check the scaled pool generation
tile_nq = 3
scaled_nq = 4 # -1.616 is ED for 4 qubits
P= 100 #steps for interpolation circuit -X-> TFIM (X+ZZ)
tile_full_bit_pool = generate_full_bit_pool(tile_nq)
tile_hamiltonian = tfim_1d_obc(tile_nq, topology=bricklayertopology(tile_nq;periodic=false))

scaled_pool = scaled_pool_generation(tile_nq, scaled_nq, tile_hamiltonian, tile_full_bit_pool; num_runs=10, run_iters=10, conv_tol=1e-2, verbose=false) 
# convert scaled pool to PauliStrings for printing
for op in scaled_pool
    println(bit_to_paulistring(op, scaled_nq)[1])
end
scaled_hamiltonian = tfim_1d_obc(scaled_nq, topology=bricklayertopology(scaled_nq;periodic=false))
interpol_circ = tfim_interpolation_circuit(scaled_nq, P)
println("Scaled adaptVQE loop:")
circuit, thetas, chosen_ops, opt_energy_gd, max_grads = adaptVQE_loop(scaled_nq, scaled_hamiltonian, scaled_pool;interpol_circ=interpol_circ, theta_init = rand(Uniform(-pi,pi)), max_iters=40, conv_tol=1e-4, verbose=false);
println("opt_energy_gd: ", opt_energy_gd)
println("max grads: " , max_grads)

Chosen operators from tile ADAPT-VQE: 17
PauliString(nqubits: 4, 1.0 * YZII)
PauliString(nqubits: 4, 1.0 * IYZI)
PauliString(nqubits: 4, 1.0 * IIYZ)
PauliString(nqubits: 4, 1.0 * YIZI)
PauliString(nqubits: 4, 1.0 * IYIZ)
PauliString(nqubits: 4, 1.0 * YIII)
PauliString(nqubits: 4, 1.0 * IYII)
PauliString(nqubits: 4, 1.0 * IIYI)
PauliString(nqubits: 4, 1.0 * IIIY)
PauliString(nqubits: 4, 1.0 * XZYI)
PauliString(nqubits: 4, 1.0 * IXZY)
PauliString(nqubits: 4, 1.0 * ZYII)
PauliString(nqubits: 4, 1.0 * IZYI)
PauliString(nqubits: 4, 1.0 * IIZY)
PauliString(nqubits: 4, 1.0 * YXII)
PauliString(nqubits: 4, 1.0 * IYXI)
PauliString(nqubits: 4, 1.0 * IIYX)
PauliString(nqubits: 4, 1.0 * YZZI)
PauliString(nqubits: 4, 1.0 * IYZZ)
PauliString(nqubits: 4, 1.0 * XIYI)
PauliString(nqubits: 4, 1.0 * IXIY)
PauliString(nqubits: 4, 1.0 * YXZI)
PauliString(nqubits: 4, 1.0 * IYXZ)
PauliString(nqubits: 4, 1.0 * ZZYI)
PauliString(nqubits: 4, 1.0 * IZZY)
PauliString(nqubits: 4, 1.0 * ZYZI)
PauliString(nqubits: 4,

In [24]:
println(opt_energy_gd[end]) #final energy per qubit
println(max_grads[end]) #final max gradient

-0.9232993324675288
1.812080235298796


### Compare to regular case (without DQA initialisation)

In [25]:
tile_nq = 3
scaled_nq = 4 # -1.616 is ED for 4 qubits
tile_full_bit_pool = generate_full_bit_pool(tile_nq)
tile_hamiltonian = tfim_1d_obc(tile_nq, topology=bricklayertopology(tile_nq;periodic=false))

scaled_pool = scaled_pool_generation(tile_nq, scaled_nq, tile_hamiltonian, tile_full_bit_pool; num_runs=10, run_iters=10, conv_tol=1e-2, verbose=false) 
# need approx. 10 runs to hit the 12 different operators 
# convert scaled pool to PauliStrings for printing
for op in scaled_pool
    println(bit_to_paulistring(op, scaled_nq)[1])
end
scaled_hamiltonian = tfim_1d_obc(scaled_nq, topology=bricklayertopology(scaled_nq;periodic=false))
println("Scaled adaptVQE loop:")
circuit, thetas, chosen_ops, opt_energy_gd, max_grads = adaptVQE_loop(scaled_nq, scaled_hamiltonian, scaled_pool;interpol_circ=nothing, theta_init = rand(Uniform(-pi,pi)), max_iters=40, conv_tol=1e-4, verbose=false);
println("opt_energy_gd: ", opt_energy_gd)
println("max grads: " , max_grads)

Chosen operators from tile ADAPT-VQE: 13
PauliString(nqubits: 4, 1.0 * ZZYI)
PauliString(nqubits: 4, 1.0 * IZZY)
PauliString(nqubits: 4, 1.0 * ZIYI)
PauliString(nqubits: 4, 1.0 * IZIY)
PauliString(nqubits: 4, 1.0 * YZZI)
PauliString(nqubits: 4, 1.0 * IYZZ)
PauliString(nqubits: 4, 1.0 * YIZI)
PauliString(nqubits: 4, 1.0 * IYIZ)
PauliString(nqubits: 4, 1.0 * YIII)
PauliString(nqubits: 4, 1.0 * IYII)
PauliString(nqubits: 4, 1.0 * IIYI)
PauliString(nqubits: 4, 1.0 * IIIY)
PauliString(nqubits: 4, 1.0 * ZYII)
PauliString(nqubits: 4, 1.0 * IZYI)
PauliString(nqubits: 4, 1.0 * IIZY)
PauliString(nqubits: 4, 1.0 * ZYZI)
PauliString(nqubits: 4, 1.0 * IZYZ)
PauliString(nqubits: 4, 1.0 * YZII)
PauliString(nqubits: 4, 1.0 * IYZI)
PauliString(nqubits: 4, 1.0 * IIYZ)
PauliString(nqubits: 4, 1.0 * YXII)
PauliString(nqubits: 4, 1.0 * IYXI)
PauliString(nqubits: 4, 1.0 * IIYX)
Scaled adaptVQE loop:
opt_energy_gd: [0.0, 0.0, -1.0, -1.0, -1.0590169943749475, -1.0590169943749475, -1.0590169943749475, -1.05901